Natural Language Processing starting : 
1. Import the lib and dataset
2. Clean the text
3. Clean the bag of words
4. Split the dataset into train and test
5. Apply any algo like Naive Bayes or something else here classification is done
6. Evaluate test set
7. Confustion Matrix

Importing the libraries

In [480]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Import the dataset
1. here tsv is there tab separated values, delimiter is tab here \t so remember it
2. and also , is not a delimitter
3. quote=3 means ignore the double quotes "

In [481]:
data=pd.read_csv('datasets/Restaurant_Reviews.tsv',delimiter='\t',quoting=3)
data.head(10)

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
5,Now I am getting angry and I want my damn pho.,0
6,Honeslty it didn't taste THAT fresh.),0
7,The potatoes were like rubber and you could te...,0
8,The fries were great too.,1
9,A great touch.,1


Here 0 --> Not Liked
and 1 --> Liked

Data Preprocessing

Clean the text, 
1. Importing the libraries

In [482]:
import re  #regular expression to subtracting the other letters
import nltk   # to download stopwords and other things
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [483]:
from nltk.corpus import stopwords  # importing stop words
from nltk.stem.porter import PorterStemmer   #converting into root words
# it reduces the complexity of the sparse matrix

2. Cleaning

But first of all basics

it is very important to notice that i have downloaded the stopwords but they are mostly irrelavant
1. that's why i have to remove them
2. but there is a problem
3. words like no,not, etc are also included in stopwords which are essential for our review 
4. therefore we have to identify those words and should remove them from the set of irrelavant words

In [484]:
#stopwords has no also so remove it
all_stopword=stopwords.words('english')
relevant_stopwords = ['no', 'none', 'nothing', 'never', 'without', 'nor', 'cannot', 'won\'t', 'doesn\'t', 'isn\'t', 'wasn\'t', 'aren\'t', 'could']
irrelevant_stopwords = [word for word in all_stopword if word not in relevant_stopwords]


In [485]:
irrelevant_stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

Function for preprocessing

In [486]:
def reveiw_preprocessing(text):           #the text is the review taken from data
    review=re.sub('[^a-zA-Z^]',' ',text)  #to remove all letters other than alphabets
    #first argument of sub is to remove the words but here ^ is used so except these words remove others
    # second argument is to replace them by space
    #third argument is the string/review
    review=review.lower()     #to convert into lower as it don't understand that Loved and loved is same
    review=review.split()      #here we splitted the sentence so that every word can be converted into their root form
    ps=PorterStemmer()
    review=[ps.stem(word) for word in review if not word in set(irrelevant_stopwords)]   #still review is a list
    # here we are stemming the word which are not in all stopword and now review only contains root words and not any stopwords
    review=' '.join(review)
    return review


NOW CREATE THE CORPUS IN OTHER WORDS THE DATASET WHICH CONTAIN THESE PREPROCESSED REVIEWS

In [487]:
corpus=[]  # we will add them one by one
for i in range(0,len(data)):
    corpus.append(reveiw_preprocessing(data['Review'][i]))

Let's check what is done till now

In [488]:
data['Review'][45]

'The only thing I did like was the prime rib and dessert section.'

In [489]:
corpus[45]

'thing like prime rib dessert section'

3. Bag of Words

In [490]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=1500)
#it has a argument of max features which we will be determining now

matrix of features(x and y)

In [491]:
x=cv.fit_transform(corpus).toarray()
#fit take all the words, transform put them in columns and to array is for 2d
y=data.iloc[:,-1].values

In [492]:
len(x)

1000

In [493]:
len(x[5])

1500

Here 1565 words are present
1. But all are not useful
2. such as desert we don't know from this word we are getting what information it is good or bad 
3. so we will again initialise the countvectoriser with max featues 1500 to include that word which is relevant

In [494]:
len(x[0])  #only 1500 words now they are relavant it is hit and trial

1500

Splitting into train and test set

In [495]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.2)

In [496]:
# Training the model with Naive bayes classification
from sklearn.naive_bayes import GaussianNB
classifier_nb=GaussianNB()
classifier_nb.fit(x_train,y_train)

GaussianNB()

Predicting the test set result

In [497]:
y_pred=classifier_nb.predict(x_test)
# print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

Evaluating the test set

In [498]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(y_test,y_pred)
accuracy=accuracy_score(y_test,y_pred)
print(f"The accuracy is {accuracy*100}")
print("Confusion Matrix\n")
print(cm)

The accuracy is 67.0
Confusion Matrix

[[48 48]
 [18 86]]


In [499]:
from sklearn.linear_model import LogisticRegression
classifier_lr=LogisticRegression()
classifier_lr.fit(x_train,y_train)
y_pred=classifier_lr.predict(x_test)
print(f"Accuracy Score of Logistic Regression :{accuracy_score(y_test,y_pred)*100}")
cm=confusion_matrix(y_test,y_pred)
print("Confusion Matrix")
print(cm)

Accuracy Score of Logistic Regression :74.5
Confusion Matrix
[[81 15]
 [36 68]]


In [500]:
from sklearn.svm import SVC

# Create a pipeline with CountVectorizer and SVC
classifier_svc=SVC(kernel='rbf')
classifier_svc.fit(x_train, y_train)
y_pred = classifier_svc.predict(x_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"SVM Accuracy: {accuracy*100}")
cm=confusion_matrix(y_test,y_pred)
print(cm)


SVM Accuracy: 71.0
[[86 10]
 [48 56]]


In [501]:
from sklearn.ensemble import RandomForestClassifier

classifier_rtc=RandomForestClassifier(n_estimators=100,random_state=42,criterion='gini')
classifier_rtc.fit(x_train,y_train)
y_pred = classifier_rtc.predict(x_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Accuracy: {accuracy*100}")


Random Forest Accuracy: 70.0


So the Best model here is Logistic Regression

Let's Predict a single result

In [502]:
your_review = str(input("Enter your review about the food and the restaurant: "))
preprocessed_review = reveiw_preprocessing(your_review)
vectorized_review = cv.transform([preprocessed_review])
result = classifier_lr.predict(vectorized_review)
print(f"Review: {your_review}")
print("Prediction:", "Positive" if result[0] == 1 else "Negative")


Review: I literally did not liked the food, the dal was burnt. Overall the food tasted terrible.. Thanks
Prediction: Negative


I AM STILL NOT SATISFIED LET'S DO MORE TESTING LET'S SEE CAN IT I IDENTIFY THEM, I TOOK 10 POSITIVE AND 10 NEGATIVE SENTENCES EACH AND AGAIN PREDICTED THEM

In [503]:
positive_sentences = [
        "The food was absolutely amazing and the service was impeccable.",
        "We had a wonderful time; the dishes were bursting with flavor.",
        "Every dish we ordered was a culinary masterpiece.",
        "Great experience overall; the staff were welcoming and the food was delicious.",
        "The ambiance was perfect, and the meal was fantastic.",
        "I loved the creative menu and the food was superb.",
        "Fantastic service and the meal was exactly what we needed.",
        "The desserts were heavenly and the main course was superb.",
        "Our dining experience was unforgettable; the food was divine.",
        "Exceptional quality and friendly service made our visit perfect."
    ]

In [504]:
result=[]
real_result=[1,1,1,1,1,1,1,1,1,1]
for i in range(0,len(positive_sentences)):
    preprocessed_positive_reviews=reveiw_preprocessing(positive_sentences[i])
    vectorized_review = cv.transform([preprocessed_positive_reviews])
    prediction=classifier_lr.predict(vectorized_review)
    result.append(prediction[0])


print(result)
print(accuracy_score(real_result,result)*100,"percent")

[1, 1, 1, 1, 1, 1, 1, 0, 0, 1]
80.0 percent


In [505]:
negative_sentences = [
        "The meal was a disappointment; the flavors were off.",
        "Service was slow and the food was mediocre at best.",
        "The restaurant was noisy and the food was bland.",
        "Not a pleasant experience; the staff were inattentive and the food was cold.",
        "I was very let down; the quality of the food was poor.",
        "The place was overrated; the food did not live up to the hype.",
        "The portions were tiny, and the taste was lacking.",
        "Unimpressed with the food; it was greasy and undercooked.",
        "The dining experience was ruined by the poor service and bad food.",
        "The food was overpriced and underwhelming; I wont be coming back."
    ]

In [506]:
result=[]
real_result=[0]*10
for i in range(0,len(negative_sentences)):
    preprocessed_negative_reviews=reveiw_preprocessing(negative_sentences[i])
    vectorized_review = cv.transform([preprocessed_negative_reviews])
    prediction=classifier_lr.predict(vectorized_review)
    result.append(prediction[0])


print(result)
print(accuracy_score(real_result,result)*100,"percent")

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
100.0 percent
